In [ ]:
import copy
import csv
import os
import warnings
from argparse import ArgumentParser

import torch
import tqdm
import yaml
from torch.utils import data
# 개별 json 라벨 파일을 이용해 학습 데이터 리스트 생성
import glob
import json
import os
from nets import nn
from utils import util
from utils.dataset import Dataset
from torch.utils import data
import numpy as np
import cv2
import random
import matplotlib.pyplot as plt
import matplotlib.pyplot as plt
import matplotlib.patches as patches
from sklearn.model_selection import train_test_split  # sklearn 사용
device=torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print("device:",device)

In [ ]:

# 파라미터 및 데이터 경로 설정
with open('utils/args.yaml', errors='ignore') as f:
    params = yaml.safe_load(f)


label_dir = '../../data/HnE_cell_detect/total_data/labels/'
image_dir = '../../data/HnE_cell_detect/total_data/images/'

label_files = sorted(glob.glob(os.path.join(label_dir, '*.json')))
filenames = []
labels = []
for label_file in label_files:
    with open(label_file) as f:
        data1 = json.load(f)
    img_path = os.path.join(image_dir, data1['file_name'])
    if os.path.exists(img_path):
        filenames.append(img_path)
        temp_labels = []
        for i in range(len(data1["cordinates"])):
            if data1["cordinates"][i][3] >50 or data1["cordinates"][i][4]>50:
                continue
            temp_labels.append(data1["cordinates"][i])
        labels.append(temp_labels)


In [ ]:
class custom_dataset(data.Dataset):
    def __init__(self, filenames, input_size, params, augment, labels=None, image_infos=None):
        self.params = params
        self.mosaic = augment
        self.augment = augment
        self.input_size = input_size
        if labels is not None:
            self.labels = labels
            self.filenames = filenames
            self.n = len(self.filenames)
            self.image_infos = image_infos if image_infos is not None else [None]*len(filenames)
        else:
            loaded = self.load_label(filenames)
            self.labels = list(loaded.values())
            self.filenames = list(loaded.keys())
            self.n = len(self.filenames)
            self.image_infos = [None]*self.n
        self.indices = range(self.n)
        self.albumentations = Albumentations()
    def __len__(self):
        return self.n*5
    def __getitem__(self, index):
        index= index % self.n
        index = self.indices[index]
        temp_label = copy.deepcopy(self.labels[index])
        
        image,crop_index=self.load_image(index)
        
        crop_y, crop_x = crop_index
        label=[]
        #y,x,h,w, to x_center,y_center,w,h
        for i in range(len(temp_label)):
            x = temp_label[i][2]
            y = temp_label[i][1]
            w = temp_label[i][4]
            h = temp_label[i][3]
            if x >= crop_x and y >= crop_y and x <= crop_x + self.input_size and y <= crop_y + self.input_size:
                temp_label[i][1] = (x+w/2 - crop_x)/ self.input_size
                temp_label[i][2] = (y+h/2 - crop_y)/ self.input_size
                temp_label[i][3] = (w) / self.input_size
                temp_label[i][4] = (h) / self.input_size
                label.append(temp_label[i])

        cls=[]
        box=[]
        for i in range(len(label)):
            cls.append(label[i][0]-1)
            box.append(label[i][1:5])
        cls=np.array(cls)
        box=np.array(box)
        nl = len(box)
        if self.augment:
            nl = len(box)  # update after albumentations

            # Flip up-down
            if random.random() < self.params['flip_ud']:
                image = np.flipud(image).copy()
                if nl:
                    box[:, 1] = 1 - box[:, 1]
            # Flip left-right
            if random.random() < self.params['flip_lr']:
                image = np.fliplr(image).copy()
                if nl:
                    box[:, 0] = 1 - box[:, 0]

        image = image.transpose((2, 0, 1))
        return torch.from_numpy(image),torch.from_numpy(cls), torch.from_numpy(box), torch.zeros(nl)

    def load_image(self, i):
        image = cv2.imread(self.filenames[i])
        image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)  # BGR -> RGB 변환
        h, w = image.shape[:2]
        r = self.input_size / min(h, w)
        
        # 이미지가 input_size보다 큰 경우 (r < 1) -> 랜덤 크롭
        if r < 1:
            # 안전하게 크롭 범위 계산
            max_h = max(0, h - self.input_size)
            max_w = max(0, w - self.input_size)
            h1 = random.randint(0, max_h) if max_h > 0 else 0
            w1 = random.randint(0, max_w) if max_w > 0 else 0
            image = image[h1:h1 + self.input_size, w1:w1 + self.input_size]
        else:
            # 이미지가 input_size보다 작거나 같은 경우 (r >= 1) -> 패딩
            h1 = 0
            w1 = 0
            pad_image = np.ones((self.input_size, self.input_size, 3), dtype=np.uint8)*255
            pad_image[:min(h,self.input_size), :min(w,self.input_size), :] = image[:min(h,self.input_size), :min(w,self.input_size), :]
            image = pad_image
        return image, (h1, w1)


    
    
class Albumentations:
    def __init__(self):
        self.transform = None
        try:
            import albumentations

            transforms = [albumentations.Blur(p=0.01),
                          albumentations.CLAHE(p=0.01),
                          albumentations.ToGray(p=0.01),
                          albumentations.MedianBlur(p=0.01)]
            self.transform = albumentations.Compose(transforms,
                                                    albumentations.BboxParams('yolo', ['class_labels']))

        except ImportError:  # package not installed, skip
            pass

    def __call__(self, image, box, cls):
        if self.transform:
            x = self.transform(image=image,
                               bboxes=box,
                               class_labels=cls)
            image = x['image']
            box = np.array(x['bboxes'])
            cls = np.array(x['class_labels'])
        return image, box, cls

split=[0.9, 0.1]
x_train,x_val,y_train,y_val=train_test_split(filenames,labels,test_size=0.1,random_state=42,shuffle=True)
train_dataset=custom_dataset(x_train,512, params, augment=True, labels=y_train)
val_dataset=custom_dataset(x_val,512, params, augment=False, labels=y_val)

In [ ]:
def collate_fn1(batch):
    samples, cls, box, indices = zip(*batch)

    cls = torch.cat(cls, dim=0)
    box = torch.cat(box, dim=0)

    new_indices = list(indices)
    for i in range(len(indices)):
        new_indices[i] += i * box.size(0)  # 각 인덱스에 배치 내 샘플 수를 곱함
    indices = torch.cat(new_indices, dim=0)

    targets = {'cls': cls,
                'box': box,
                'idx': indices}
    return torch.stack(samples, dim=0), targets


# 모델 및 파라미터 준비
model = nn.yolo_v11_m(len(params['names'])).to(device)
optimizer = torch.optim.SGD(util.set_params(model, params['weight_decay']),
                            params['min_lr'], params['momentum'], nesterov=True)
criterion = util.ComputeLoss(model, params)

# 데이터셋 및 데이터로드 (안전한 함수 사용)
batch_size = 4
# 안전하게 데이터로더 생성하는 함수
def create_safe_loader(dataset, batch_size, is_train=True):
    """
    배치 크기에 맞게 데이터셋을 조정하여 안전하게 데이터로더를 생성하는 함수
    """
    dataset_size = len(dataset)
    
    # 배치 크기가 데이터셋 크기보다 큰 경우 배치 크기 조정
    if dataset_size < batch_size:
        print(f"경고: 데이터셋 크기({dataset_size})가 배치 크기({batch_size})보다 작습니다. 배치 크기를 {dataset_size}로 조정합니다.")
        actual_batch_size = max(1, dataset_size)
    else:
        actual_batch_size = batch_size
    
    # 데이터셋이 배치 크기로 나누어 떨어지는지 확인
    if dataset_size % actual_batch_size != 0:
        print(f"참고: 데이터셋 크기({dataset_size})가 배치 크기({actual_batch_size})로 나누어 떨어지지 않습니다.")
        print(f"마지막 배치는 {dataset_size % actual_batch_size}개의 샘플을 포함합니다.")
    
    # 데이터로더 생성
    loader = torch.utils.data.DataLoader(
        dataset, 
        batch_size=actual_batch_size, 
        shuffle=is_train,
        num_workers=4,
        collate_fn=collate_fn1,
        drop_last=(not is_train)  # 훈련 시에는 마지막 배치 유지, 검증 시에는 마지막 배치 제외
    )
    
    return loader, actual_batch_size
# 안전하게 데이터로더 생성
loader, train_batch_size = create_safe_loader(train_dataset, batch_size, is_train=True)
val_loader, val_batch_size = create_safe_loader(val_dataset, 1, is_train=False)

print(f"최종 훈련 배치 크기: {train_batch_size}")
print(f"최종 검증 배치 크기: {val_batch_size}")



In [ ]:
# val_loader를 사용한 배치 단위 시각화
def visualize_val_loader_samples(val_loader, num_batches=4):
    """
    val_loader에서 배치 단위로 샘플을 가져와서 시각화
    """
    colors = ['red', 'blue', 'green', 'orange', 'purple', 'brown', 'pink', 'gray', 'olive', 'cyan']
    class_names = {0:"Neutrophil", 1:"Epithelial", 2:"Lymphocyte", 
                   3:"Plasma", 4:"Eosinophil", 5:"Connective tissue"}
    
    fig, axes = plt.subplots(2, 2, figsize=(15, 15))
    axes = axes.flatten()
    
    batch_count = 0
    for batch_idx, (images, targets) in enumerate(val_loader):
        if batch_count >= num_batches:
            break
            
        # 첫 번째 이미지만 시각화 (배치 크기가 1이므로)
        image_tensor = images[0]  # (C, H, W)
        cls_tensor = targets['cls']
        box_tensor = targets['box']
        
        # 텐서를 numpy로 변환 및 이미지 복원
        image = image_tensor.permute(1, 2, 0).numpy()  # (C, H, W) -> (H, W, C)
       # 0-1 범위로 클리핑
        
        cls = cls_tensor.numpy() if len(cls_tensor) > 0 else []
        boxes = box_tensor.numpy() if len(box_tensor) > 0 else []
        
        ax = axes[batch_count]
        ax.imshow(image)
        ax.set_title(f'Val Loader Batch {batch_idx}', fontsize=12)
        
        # 바운딩 박스 그리기 (YOLO 형식: 정규화된 중심점 좌표)
        class_counts = {}
        h_img, w_img = image.shape[:2]
        
        for i, (class_id, box) in enumerate(zip(cls, boxes)):
            if len(box) < 4:
                continue
                
            # YOLO 형식: [y_center, x_center, height, width] (정규화됨)
            x_center_norm, y_center_norm, w_norm, h_norm = box
            
            # 정규화된 좌표를 절대 좌표로 변환
            x_center = x_center_norm * w_img
            y_center = y_center_norm * h_img
            w_abs = w_norm * w_img
            h_abs = h_norm * h_img
            
            # 좌상단 좌표 계산
            x_abs = x_center - w_abs / 2
            y_abs = y_center - h_abs / 2
            
            # 클래스별 개수 세기
            class_name = class_names.get(int(class_id), f"Class_{int(class_id)}")
            class_counts[class_name] = class_counts.get(class_name, 0) + 1
            
            # 바운딩 박스 생성
            rect = patches.Rectangle((x_abs, y_abs), w_abs, h_abs, 
                                   linewidth=2, 
                                   edgecolor=colors[int(class_id)], 
                                   facecolor='none')
            ax.add_patch(rect)
            
            # 클래스 라벨 텍스트 추가
            
            # 중심점 표시 (YOLO 특징)
            ax.plot(x_center, y_center, 'o', color=colors[int(class_id)], markersize=3)
        
        # 클래스별 개수 정보 표시
        info_text = '\n'.join([f'{name}: {count}' for name, count in class_counts.items()])
        if info_text:
            ax.text(0.02, 0.98, info_text, 
                   transform=ax.transAxes, 
                   verticalalignment='top',
                   bbox=dict(boxstyle="round,pad=0.5", facecolor='lightblue', alpha=0.8),
                   fontsize=10)
        
        ax.axis('off')
        batch_count += 1
    
    # 빈 서브플롯 숨기기
    for idx in range(batch_count, len(axes)):
        axes[idx].axis('off')
    
    plt.tight_layout()
    plt.show()

print("\n🖼️  Val Loader 배치 샘플 시각화 (DataLoader에서 직접 가져온 데이터)")
visualize_val_loader_samples(val_loader, num_batches=4)

In [ ]:
def visualize_ground_truth_and_prediction_separately(model, dataset, idx=0, conf_threshold=0.2, iou_threshold=0.3, epoch=None, save_dir=None):
    """실제 라벨과 예측 라벨을 subplot으로 좌우에 표시하는 함수"""
    if len(dataset) <= idx:
        print(f"경고: 데이터셋이 비어 있거나 idx {idx}가 데이터셋 크기({len(dataset)})보다 큽니다.")
        return
    
    model.eval()
    img, cls, box, _ = dataset[idx]
    
    # 하나의 figure에 2개의 subplot 생성 (1행 2열)
    fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(16, 8))
    img = img.cpu()/255.
    # Subplot 1: Ground Truth (실제 라벨)
    ax1.imshow(img.permute(1, 2, 0).cpu().numpy())
    class_names = {
            0: "Neutrophil",
            1: "Epithelial",
            2: "Lymphocyte",
            3: "Plasma",
            4: "Eosinophil",
            5: "Connective tissue"
        }
    for i in range(len(cls)):
        class_id = cls[i].item()
        x_center, y_center, w, h = box[i].tolist()
        
        x = (x_center - w/2) * img.shape[2]
        y = (y_center - h/2) * img.shape[1]
        w_box = w * img.shape[2]
        h_box = h * img.shape[1]
        if class_id == 0: #Neutrophil
            color = 'orange'
        elif class_id == 1: #Epithelial
            color = 'green'
        elif class_id == 2: #Lymphocyte
            color = 'red'
        elif class_id == 3: #Plasma
            color = 'skyblue'
        elif class_id == 4: #Eosinophil
            color = 'blue'
        elif class_id == 5: #Connective tissue
            color = 'yellow'
        # 중심점 표시
        # 중심점 좌표 계산
        center_x = int(x + w_box / 2)
        center_y = int(y + h_box / 2)

        ax1.scatter(center_x, center_y, facecolors='none',  s=20, marker='o', edgecolors=color, linewidths=1)

    gt_title = f'Ground Truth '
    if epoch is not None:
        gt_title += f' - Epoch {epoch}'
    ax1.set_title(gt_title, fontsize=16, fontweight='bold')
    ax1.axis('off')
    
    # Subplot 2: Model Prediction (예측 라벨)
    ax2.imshow(img.permute(1, 2, 0).cpu().numpy())
    
    prediction_count = 0
    with torch.no_grad():
        img_input = img.unsqueeze(0).to(device)
        with torch.amp.autocast('cuda'):
            pred = model(img_input)

        # NMS 적용
        results = util.non_max_suppression(pred, confidence_threshold=conf_threshold, iou_threshold=iou_threshold)
        if len(results[0]) > 0:
            for *xyxy, conf, cls_id in results[0]:
                x1, y1, x2, y2 = xyxy
                x1, y1, x2, y2 = x1.item(), y1.item(), x2.item(), y2.item()
                w_pred = x2 - x1
                h_pred = y2 - y1

                if cls_id.item() == 0: #Neutrophil
                    color = 'orange'
                elif cls_id.item() == 1: #Epithelial
                    color = 'yellow'
                elif cls_id.item() == 2: #Lymphocyte
                    color = 'blue'
                elif cls_id.item() == 3: #Plasma
                    color = 'skyblue'
                elif cls_id.item() == 4: #Eosinophil
                    color = 'purple'
                elif cls_id.item() == 5: #Connective tissue
                    color = 'orange'
                # 중심점 표시
                center_x = (x1 + x2)//2
                center_y = (y1 + y2)//2
                ax2.scatter(center_x, center_y, facecolors='none',  s=20, marker='o', edgecolors=color, linewidths=1)

                prediction_count += 1
        
        if prediction_count == 0:
            ax2.text(img.shape[2]//2, img.shape[1]//2, 'No Predictions', 
                     fontsize=20, color='white', ha='center', va='center',
                     bbox=dict(facecolor='red', alpha=0.8, pad=10))
    
    pred_title = f'Model Prediction - {prediction_count} detections'
    if epoch is not None:
        pred_title += f' - Epoch {epoch}'
    ax2.set_title(pred_title, fontsize=16, fontweight='bold')
    ax2.axis('off')
    
    # 전체 figure 제목 설정
    if epoch is not None:
        fig.suptitle(f'Validation Comparison - Epoch {epoch}, Sample {idx+1}', 
                     fontsize=18, fontweight='bold', y=0.95)
    
    # 범례 추가
    legend_elements = [
        patches.Patch(color='orange', label='Neutrophil'),
        patches.Patch(color='green', label='Epithelial'),
        patches.Patch(color='red', label='Lymphocyte'),
        patches.Patch(color='skyblue', label='Plasma'),
        patches.Patch(color='purple', label='Eosinophil'),
        patches.Patch(color='yellow', label='Connective tissue'),
    ]
    fig.legend(handles=legend_elements, loc='lower center', ncol=3, 
               bbox_to_anchor=(0.5, 0.02), fontsize=12)
    
    # 레이아웃 조정
    plt.tight_layout()
    plt.subplots_adjust(bottom=0.15, top=0.85)
    
    # 저장
    if save_dir and epoch:
        save_path = os.path.join(save_dir, f'validation_comparison_epoch_{epoch}.png')
        plt.savefig(save_path, dpi=150, bbox_inches='tight')
        print(f"✅ 비교 이미지 저장: {save_path}")
    
    # plt.show()
    plt.clf()
    

In [ ]:
from utils.valid import compute_validation_metrics, compute_validation_metrics_with_kappa, get_kappa_interpretation

from utils.valid import plot_training_progress


# main.py의 train 함수를 참조한 개선된 학습 루프
train_losses = []
val_maps = []
val_precisions = []
val_recalls = []
val_map50s = []
val_kappas = []  # Cohen's Kappa 추가
epochs = 10000

# 체크포인트 저장을 위한 디렉토리 생성
save_dir = '../../model/HnE_cell_detection/yolov11/'
os.makedirs(save_dir, exist_ok=True)
#체크포인트 불러오기 
checkpoint_path = os.path.join(save_dir, 'best_model.pt')
if os.path.exists(checkpoint_path):
    checkpoint = torch.load(checkpoint_path, map_location=device,weights_only=False)
    model.load_state_dict(checkpoint['model_state_dict'])
    
# main.py 스타일의 설정들
best_map = 0
accumulate = max(round(64 / batch_size), 1)  # gradient accumulation steps
amp_scale = torch.amp.GradScaler()  # mixed precision scaler

print(f"Gradient accumulation steps: {accumulate}")

for epoch in range(epochs):
    # 훈련
    model.train()
    
    # main.py 스타일의 평균 손실 추적
    avg_box_loss = util.AverageMeter()
    avg_cls_loss = util.AverageMeter()
    avg_dfl_loss = util.AverageMeter()
    
    train_pbar = tqdm.tqdm(enumerate(loader), total=len(loader), desc=f'Epoch {epoch+1}/{epochs} Training')
    
    optimizer.zero_grad()
    
    for i, (images, targets) in train_pbar:
        step = i + len(loader) * epoch
        images = images.to(device).float() / 255.
        
        # Forward pass with mixed precision
        with torch.amp.autocast('cuda'):
            outputs = model(images)
            loss_box, loss_cls, loss_dfl = criterion(outputs, targets)
        
        # 평균 손실 업데이트
        avg_box_loss.update(loss_box.item(), images.size(0))
        avg_cls_loss.update(loss_cls.item(), images.size(0))
        avg_dfl_loss.update(loss_dfl.item(), images.size(0))
        
        # Loss scaling for gradient accumulation
        loss_box *= batch_size  # loss scaled by batch_size
        loss_cls *= batch_size  # loss scaled by batch_size  
        loss_dfl *= batch_size  # loss scaled by batch_size
        
        total_loss = loss_box + loss_cls + loss_dfl
        
        # Backward pass with gradient scaling
        amp_scale.scale(total_loss).backward()
        
        # Gradient accumulation 및 optimization
        if step % accumulate == 0:
            # Gradient clipping 및 optimization
            amp_scale.step(optimizer)
            amp_scale.update()
            optimizer.zero_grad()
        
        # GPU 메모리 동기화
        torch.cuda.synchronize()
        
        # 진행률 표시 업데이트 (main.py 스타일)
        memory = f'{torch.cuda.memory_reserved() / 1E9:.4g}G'
        s = f'Memory: {memory} | Box: {avg_box_loss.avg:.3f} | Cls: {avg_cls_loss.avg:.3f} | DFL: {avg_dfl_loss.avg:.3f}'
        train_pbar.set_description(f'Epoch {epoch+1}/{epochs} | {s}')
    
    # 에폭 평균 손실 계산
    avg_train_loss = avg_box_loss.avg + avg_cls_loss.avg + avg_dfl_loss.avg
    train_losses.append(avg_train_loss)
    
    # 검증 (Cohen's Kappa 포함)
    precision, recall, map50, mean_ap, kappa = compute_validation_metrics_with_kappa(
        model, val_loader, device, params
    )
    val_maps.append(mean_ap)
    val_precisions.append(precision)
    val_recalls.append(recall)
    val_map50s.append(map50)
    val_kappas.append(kappa)
    
    # F1-score 계산
    f1_score = 2 * (precision * recall) / (precision + recall) if (precision + recall) > 0 else 0
    
    # 결과 출력 (Cohen's Kappa 포함)
    print(f"\nEpoch {epoch+1}/{epochs} Results:")
    print(f"  Train Loss - Box: {avg_box_loss.avg:.4f}, Cls: {avg_cls_loss.avg:.4f}, DFL: {avg_dfl_loss.avg:.4f}, Total: {avg_train_loss:.4f}")
    print(f"  Validation - Precision: {precision:.4f}, Recall: {recall:.4f}, F1-score: {f1_score:.4f}")
    print(f"  mAP@0.5: {map50:.4f}, mAP@0.5:0.95: {mean_ap:.4f}")
    print(f"  Cohen's Kappa: {kappa:.4f} ({get_kappa_interpretation(kappa)})")
    print("-" * 80)
    
    # 베스트 모델 저장 (mAP 기준)
    if mean_ap > best_map:
        best_map = mean_ap
        save_checkpoint = {
            'epoch': epoch,
            'model_state_dict': model.state_dict(),
            'optimizer_state_dict': optimizer.state_dict(),
            'amp_scale_state_dict': amp_scale.state_dict(),
            'train_loss': avg_train_loss,
            'box_loss': avg_box_loss.avg,
            'cls_loss': avg_cls_loss.avg,
            'dfl_loss': avg_dfl_loss.avg,
            'map': mean_ap,
            'map50': map50,
            'precision': precision,
            'recall': recall,
            'f1_score': f1_score,
            'kappa': kappa,
            'params': params
        }
        torch.save(save_checkpoint, os.path.join(save_dir, 'best_model.pt'))
        print(f"🎉 새로운 베스트 모델 저장! mAP: {mean_ap:.4f}, Kappa: {kappa:.4f}")
    
    # 최신 모델도 저장 (main.py 스타일)
    last_checkpoint = {
        'epoch': epoch,
        'model_state_dict': model.state_dict(),
        'optimizer_state_dict': optimizer.state_dict(),
        'amp_scale_state_dict': amp_scale.state_dict(),
        'train_loss': avg_train_loss,
        'box_loss': avg_box_loss.avg,
        'cls_loss': avg_cls_loss.avg,
        'dfl_loss': avg_dfl_loss.avg,
        'map': mean_ap,
        'map50': map50,
        'precision': precision,
        'recall': recall,
        'f1_score': f1_score,
        'kappa': kappa,
        'params': params
    }
    torch.save(last_checkpoint, os.path.join(save_dir, 'last_model.pt'))
    
    # 100 에폭마다 학습 진행 그래프 생성 및 저장
    if (epoch + 1) % 100 == 0:
        try:
            print(f"\n📊 Epoch {epoch+1} - 학습 진행 상황 그래프 생성 중...")
            plot_training_progress(train_losses, val_maps, val_precisions, val_recalls, val_map50s, epoch+1, save_dir)
        except Exception as e:
            print(f"그래프 생성 중 오류: {e}")
    
    # 개선된 검증 이미지 시각화 (매 10 에폭마다) - 실제 라벨과 예측 라벨을 별도 figure로 표시
    if (epoch + 1) % 10 == 0:
        try:
            # 여러 샘플에 대해 시각화
            num_samples = 1 # 샘플 수를 1개
            for sample_idx in range(num_samples):
                print(f"\n📊 Epoch {epoch+1} - 검증 샘플 {sample_idx+1}/{num_samples}:")
                print("=" * 60)
                
                # 실제 라벨과 예측 라벨을 별도 figure로 표시
                sample_idx = random.randint(0, len(val_dataset)-1)
                visualize_ground_truth_and_prediction_separately(
                    model, val_dataset, idx=sample_idx, 
                    epoch=epoch+1, save_dir=save_dir
                )
                
        except Exception as e:
            print(f"시각화 중 오류: {e}")

print("🎯 학습 완료!")
print(f"최종 베스트 mAP: {best_map:.4f}")
print(f"모델 저장 위치: {save_dir}")
print(f"베스트 모델: {os.path.join(save_dir, 'best_model.pt')}")
print(f"최신 모델: {os.path.join(save_dir, 'last_model.pt')}")

# 최종 성능 요약
if val_kappas:
    final_kappa = val_kappas[-1]
    final_map = val_maps[-1]
    final_precision = val_precisions[-1]
    final_recall = val_recalls[-1]
    final_f1 = 2 * (final_precision * final_recall) / (final_precision + final_recall) if (final_precision + final_recall) > 0 else 0
    
    print(f"\n📊 최종 성능 요약:")
    print(f"  mAP@0.5:0.95: {final_map:.4f}")
    print(f"  Cohen's Kappa: {final_kappa:.4f} ({get_kappa_interpretation(final_kappa)})")
    print(f"  F1-score: {final_f1:.4f}")
    print(f"  Precision: {final_precision:.4f}")
    print(f"  Recall: {final_recall:.4f}")